# 05_Experiment_Gridtracer

In this file we ran the experiment to replicate the GridTracer results.
Results were visualized in script 06_Results_Gridtracer.

### Sources used 

We use the EvalHook and MyTrainer function explain in  https://eidos-ai.medium.com/training-on-detectron2-with-a-validation-set-and-plot-loss-on-it-to-avoid-overfitting-6449418fbf4e and documented in  https://gist.github.com/ortegatron/c0dad15e49c2b74de8bb09a5615d9f6b to allow calculation of the validation loss. 
We adapt it using a hook addtion taken from https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/287023 to allow for model saving. 

We have further done abstractions on https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5 provided by the Detectron2 Team.

In [1]:
# we set a counter to register the same dataset again under a different name when we are debugging
global regis_num 
regis_num = 0

In [2]:
#!pip install pyyaml==5.1
#!pip uninstall torch
#!yes | pip uninstall detectron2
#!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
    
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Found existing installation: detectron2 0.6+cu111
Uninstalling detectron2-0.6+cu111:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/detectron2-0.6+cu111.dist-info/*
    /usr/local/lib/python3.8/dist-packages/detectron2/*
    /usr/local/lib/python3.8/dist-packages/tools/*
Proceed (y/n)?   Successfully uninstalled detectron2-0.6+cu111
yes: standard output: Broken pipe
Looking in links: https://download.pytorch.org/whl/torch_stable.html
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
  Using cached https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/detectron2-0.6%2Bcu111-cp38-cp38-linux_x86_64.whl (6.9 MB)


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
torch:  1.9 ; cuda:  cu111
detectron2: 0.6


In [3]:
#!pip install datetime

In [3]:
import os
import sys
from datetime import datetime

In [4]:
#os.environ['CUDA_VISIBLE_DEVICES'] = "2"

In [5]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data import build_detection_train_loader
from detectron2.data import DatasetMapper
from detectron2.data import DatasetCatalog


from detectron2.config import get_cfg
from detectron2 import model_zoo

In [6]:
import json

In [7]:
# Error Message while training: module 'distutils' has no attribute 'version'
#!python3 -m pip install setuptools==58.2.0
# then restart the kernel

In [8]:
# Function to register our datasets 


from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

#dataset_name_train = "gridtacer_train"
#dataset_name_val = "gridtracer_val"

def register(name, path_labels, path_data):

    # Check if dataset name already exists in catalog
    if name in DatasetCatalog.list():
        # Remove existing dataset from catalog
        DatasetCatalog.remove(name)
        MetadataCatalog.remove(name)

    # Register your dataset using register_coco_instances
    register_coco_instances(name, {}, path_labels, path_data)

# if it throws an error that name is already registered, just change the name... 
#register(dataset_name_train, "/workspace/data/data_small_sample/data_sample_030/train/labels_train.json", "/workspace/data/data_small_sample/data_sample_030/train/data")
#register(dataset_name_val, "/workspace/data/data_small_sample/data_sample_030/val/labels_val.json", "/workspace/data/data_small_sample/data_sample_030/val/data")

In [9]:
base_path = "/workspace/scripts"

os.chdir(base_path)

In [10]:
# Taken from https://eidos-ai.medium.com/training-on-detectron2-with-a-validation-set-and-plot-loss-on-it-to-avoid-overfitting-6449418fbf4e
# https://gist.github.com/ortegatron/c0dad15e49c2b74de8bb09a5615d9f6b

from EvalLossHook import LossEvalHook


class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)
                     
    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            self.cfg.TEST.EVAL_PERIOD,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg,True)
            )
        ))
        return hooks

    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg)

In [26]:
# function to train the detectron model for different resolutions 
# in the Gridtracer Experiments we only use resolution 030 therefore we hardcoded the output directory
# Output directories were hardcoded according to hyperparameter settings "model_pre-training-schedule_learning-rate_batchsize"

def train_detectron_standard(dataset_name_train, dataset_name_val, resolution):
    
    now = datetime.now() 
    output_dir = f"/workspace/output/gridtracer/output_resnet101_3x_lr0.001_bs5_10"

    cfg = get_cfg()
    
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
    cfg.OUTPUT_DIR = output_dir

    cfg.DATASETS.TRAIN = (dataset_name_train)
    cfg.DATASETS.TEST = (dataset_name_val,)

    cfg.DATALOADER.NUM_WORKERS = 4

    cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[8, 16, 32, 64, 128]]
    cfg.SOLVER.MAX_ITER = 50000
    cfg.TEST.EVAL_PERIOD = 2500
    cfg.SOLVER.STEPS = (10000, 20000, 30000, 40000)
    cfg.SOLVER.BASE_LR = 0.001
    cfg.SOLVER.IMS_PER_BATCH = 5
   
    cfg.TEST.DETECTIONS_PER_IMAGE = 30
    

    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    
    trainer = MyTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()
    
    predictor = DefaultPredictor(cfg)
    
    return cfg, trainer, predictor

In [24]:
## some different experiments were done with Resnet50 or Resnet101 and the different pre-trained versions of them

# 1. Experiment mit 1x: 40.000 iterations, learning rate 0.001 steps 15000,17000,20000,30000
# 2. Experiment mit 3x: 15.000 iterations, learning rate 0.001 steps 15000,17000,20000,30000
# 3. Experiment mit 1x: 15.000 iterations, learning rate 0.01 steps 9000,10000,12000
# 4. Experiment mit 3x 101: 15.000 iterations, learning rate 0.001 steps 3000,7000,10000,12000

In [ ]:
# train the model for resolution 030

resolution_set = ['030']

for res in resolution_set:

    #define the names 
    dataset_name_train = "tower_train_" + str(res) + "1"
    dataset_name_val = "tower_val_" + str(res) + "1"

    try: 
        # if it throws an error that name is already registered, just change the name... 
        #register(dataset_name_train, {}, "/workspace/data/labels_train.json", "/workspace/data/data_"+str(res)+"/train/data")
        #register(dataset_name_val, {}, "/workspace/data/labels_val.json", "/workspace/data/data_"+str(res)+"/val/data")

        register(dataset_name_train, "/workspace/data/data_small_sample/labels_tower_train.json", "/workspace/data/data_small_sample/data_sample_030/train/data")
        register(dataset_name_val, "/workspace/data/data_small_sample/labels_tower_val.json", "/workspace/data/data_small_sample/data_sample_030/val/data")
        
        #MetadataCatalog.get(dataset_name_train).set(thing_classes=["distribution", "transmission"])
        #MetadataCatalog.get(dataset_name_val).set(thing_classes=["distribution", "transmission"])
    
    except: 
        regis_num += 1
        dataset_name_train += str(regis_num)
        dataset_name_val += str(regis_num)

        # if it throws an error that name is already registered, just change the name... 
        #register_coco_instances(dataset_name_train, {}, "/workspace/data/labels_train.json", "/workspace/data/data_"+str(res)+"/train/data")
        #register_coco_instances(dataset_name_val, {}, "/workspace/data/labels_val.json",  "/workspace/data/data_"+str(res)+"/val/data")
        
        register(dataset_name_train, "/workspace/data/data_small_sample/labels_tower_train.json", "/workspace/data/data_small_sample/data_sample_030/train/data")
        register(dataset_name_val, "/workspace/data/data_small_sample/labels_tower_val.json", "/workspace/data/data_small_sample/data_sample_030/val/data")
        #MetadataCatalog.get(dataset_name_train).set(thing_classes=["distribution", "transmission"])
        #MetadataCatalog.get(dataset_name_val).set(thing_classes=["distribution", "transmission"])

    configuration, train_obj, predictor_obj = train_detectron_standard(dataset_name_train, dataset_name_val, res)


[04/05 11:32:00 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

WARNING [04/05 11:32:00 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/05 11:32:00 d2.data.datasets.coco]: Loaded 3927 images in COCO format from /workspace/data/data_small_sample/labels_tower_train.json
[04/05 11:32:00 d2.data.build]: Removed 0 images with no usable annotations. 3927 images left.
[04/05 11:32:01 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[04/05 11:32:01 d2.data.build]: Using training sampler TrainingSampler
[04/05 11:32:01 d2.data.common]: Serializing 3927 elements to byte tensors and concatenating them all ...
[04/05 11:32:01 d2.data.common]: Serialized dataset takes 1.49 MiB
[04/05 11:32:01 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, 

[04/05 11:32:01 d2.checkpoint.c2_model_loading]: Renaming Caffe2 weights ......
[04/05 11:32:01 d2.checkpoint.c2_model_loading]: Following weights matched with submodule backbone.bottom_up:
| Names in Model    | Names in Checkpoint       | Shapes                                          |
|:------------------|:--------------------------|:------------------------------------------------|
| res2.0.conv1.*    | res2_0_branch2a_{bn_*,w}  | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| res2.0.conv2.*    | res2_0_branch2b_{bn_*,w}  | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| res2.0.conv3.*    | res2_0_branch2c_{bn_*,w}  | (256,) (256,) (256,) (256,) (256,64,1,1)        |
| res2.0.shortcut.* | res2_0_branch1_{bn_*,w}   | (256,) (256,) (256,) (256,) (256,64,1,1)        |
| res2.1.conv1.*    | res2_1_branch2a_{bn_*,w}  | (64,) (64,) (64,) (64,) (64,256,1,1)            |
| res2.1.conv2.*    | res2_1_branch2b_{bn_*,w}  | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| res2.1.c

Some model parameters or buffers are not found in the checkpoint:
backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
The checkpoint state_dict contains keys that are not used by the model:
  fc1000.{bias, weight}


[04/05 11:32:01 d2.engine.train_loop]: Starting training from iteration 0
[04/05 11:32:14 d2.utils.events]:  eta: 2:36:46  iter: 19  total_loss: 2.639  loss_cls: 1.868  loss_box_reg: 0.001251  loss_rpn_cls: 0.6861  loss_rpn_loc: 0.06344  time: 0.6258  data_time: 0.0994  lr: 5.9943e-05  max_mem: 6209M
[04/05 11:32:26 d2.utils.events]:  eta: 2:35:38  iter: 39  total_loss: 0.8426  loss_cls: 0.08139  loss_box_reg: 0.003391  loss_rpn_cls: 0.6662  loss_rpn_loc: 0.0779  time: 0.6048  data_time: 0.0405  lr: 0.00011988  max_mem: 6209M
[04/05 11:32:38 d2.utils.events]:  eta: 2:29:31  iter: 59  total_loss: 0.8166  loss_cls: 0.1041  loss_box_reg: 0.005512  loss_rpn_cls: 0.61  loss_rpn_loc: 0.06885  time: 0.5964  data_time: 0.0402  lr: 0.00017982  max_mem: 6209M
[04/05 11:32:50 d2.utils.events]:  eta: 2:30:00  iter: 79  total_loss: 0.7139  loss_cls: 0.09535  loss_box_reg: 0.008183  loss_rpn_cls: 0.5391  loss_rpn_loc: 0.07179  time: 0.5970  data_time: 0.0272  lr: 0.00023976  max_mem: 6209M
[04/05 11

...

[04/05 13:32:40 d2.engine.defaults]: Evaluation results for tower_val_0301 in csv format:
[04/05 13:32:40 d2.evaluation.testing]: copypaste: Task: bbox
[04/05 13:32:40 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[04/05 13:32:40 d2.evaluation.testing]: copypaste: 11.5396,38.1959,3.5473,11.0478,13.9911,27.9462
[04/05 13:35:59 d2.utils.events]:  eta: 0:43:45  iter: 9999  total_loss: 0.4229  loss_cls: 0.1377  loss_box_reg: 0.2574  loss_rpn_cls: 0.00846  loss_rpn_loc: 0.02053  validation_loss: 0.7135  time: 0.5706  data_time: 0.0066  lr: 0.003  max_mem: 6209M
[04/05 13:36:11 d2.utils.events]:  eta: 0:43:34  iter: 10019  total_loss: 0.4755  loss_cls: 0.1387  loss_box_reg: 0.2927  loss_rpn_cls: 0.00747  loss_rpn_loc: 0.02099  validation_loss: 0.7135  time: 0.5706  data_time: 0.0067  lr: 0.0003  max_mem: 6209M
[04/05 13:36:23 d2.utils.events]:  eta: 0:43:24  iter: 10039  total_loss: 0.4802  loss_cls: 0.1351  loss_box_reg: 0.2842  loss_rpn_cls: 0.007979  loss_rpn_loc: 0.03099  v

[04/05 13:42:11 d2.utils.events]:  eta: 0:38:08  iter: 10639  total_loss: 0.4096  loss_cls: 0.1185  loss_box_reg: 0.2396  loss_rpn_cls: 0.007046  loss_rpn_loc: 0.02566  validation_loss: 0.7135  time: 0.5712  data_time: 0.0076  lr: 0.0003  max_mem: 6209M
[04/05 13:42:23 d2.utils.events]:  eta: 0:37:58  iter: 10659  total_loss: 0.3897  loss_cls: 0.1186  loss_box_reg: 0.2443  loss_rpn_cls: 0.008428  loss_rpn_loc: 0.01823  validation_loss: 0.7135  time: 0.5712  data_time: 0.0073  lr: 0.0003  max_mem: 6209M
[04/05 13:42:34 d2.utils.events]:  eta: 0:37:47  iter: 10679  total_loss: 0.412  loss_cls: 0.1285  loss_box_reg: 0.2568  loss_rpn_cls: 0.004758  loss_rpn_loc: 0.02165  validation_loss: 0.7135  time: 0.5712  data_time: 0.0072  lr: 0.0003  max_mem: 6209M
[04/05 13:42:46 d2.utils.events]:  eta: 0:37:37  iter: 10699  total_loss: 0.4359  loss_cls: 0.1351  loss_box_reg: 0.2728  loss_rpn_cls: 0.00533  loss_rpn_loc: 0.01839  validation_loss: 0.7135  time: 0.5712  data_time: 0.0073  lr: 0.0003  m

[04/05 13:48:34 d2.utils.events]:  eta: 0:36:01  iter: 11299  total_loss: 0.3632  loss_cls: 0.1059  loss_box_reg: 0.2325  loss_rpn_cls: 0.006214  loss_rpn_loc: 0.02004  validation_loss: 0.7135  time: 0.5717  data_time: 0.0067  lr: 0.0003  max_mem: 6209M
[04/05 13:48:45 d2.utils.events]:  eta: 0:35:49  iter: 11319  total_loss: 0.4048  loss_cls: 0.1217  loss_box_reg: 0.2507  loss_rpn_cls: 0.005429  loss_rpn_loc: 0.025  validation_loss: 0.7135  time: 0.5716  data_time: 0.0067  lr: 0.0003  max_mem: 6209M
[04/05 13:48:56 d2.utils.events]:  eta: 0:35:37  iter: 11339  total_loss: 0.3799  loss_cls: 0.1134  loss_box_reg: 0.2332  loss_rpn_cls: 0.004239  loss_rpn_loc: 0.02537  validation_loss: 0.7135  time: 0.5717  data_time: 0.0067  lr: 0.0003  max_mem: 6209M
[04/05 13:49:08 d2.utils.events]:  eta: 0:35:25  iter: 11359  total_loss: 0.3911  loss_cls: 0.1181  loss_box_reg: 0.2451  loss_rpn_cls: 0.006679  loss_rpn_loc: 0.02125  validation_loss: 0.7135  time: 0.5717  data_time: 0.0069  lr: 0.0003  m

[04/05 13:54:56 d2.utils.events]:  eta: 0:29:33  iter: 11959  total_loss: 0.3907  loss_cls: 0.1143  loss_box_reg: 0.2298  loss_rpn_cls: 0.005759  loss_rpn_loc: 0.02237  validation_loss: 0.7135  time: 0.5721  data_time: 0.0065  lr: 0.0003  max_mem: 6209M
[04/05 13:55:07 d2.utils.events]:  eta: 0:29:24  iter: 11979  total_loss: 0.3659  loss_cls: 0.1051  loss_box_reg: 0.2242  loss_rpn_cls: 0.005934  loss_rpn_loc: 0.03189  validation_loss: 0.7135  time: 0.5721  data_time: 0.0066  lr: 0.0003  max_mem: 6209M
[04/05 13:55:19 d2.utils.events]:  eta: 0:29:14  iter: 11999  total_loss: 0.3722  loss_cls: 0.1064  loss_box_reg: 0.2269  loss_rpn_cls: 0.004096  loss_rpn_loc: 0.0203  validation_loss: 0.7135  time: 0.5721  data_time: 0.0065  lr: 0.0003  max_mem: 6209M
[04/05 13:55:31 d2.utils.events]:  eta: 0:29:02  iter: 12019  total_loss: 0.3574  loss_cls: 0.1073  loss_box_reg: 0.2231  loss_rpn_cls: 0.005321  loss_rpn_loc: 0.02562  validation_loss: 0.7135  time: 0.5721  data_time: 0.0068  lr: 0.0003  

[04/05 14:00:25 d2.evaluation.evaluator]: Inference done 182/1878. Dataloading: 0.0010 s/iter. Inference: 0.0870 s/iter. Eval: 0.0001 s/iter. Total: 0.0883 s/iter. ETA=0:02:29
[04/05 14:00:30 d2.evaluation.evaluator]: Inference done 239/1878. Dataloading: 0.0010 s/iter. Inference: 0.0873 s/iter. Eval: 0.0001 s/iter. Total: 0.0886 s/iter. ETA=0:02:25
[04/05 14:00:35 d2.evaluation.evaluator]: Inference done 296/1878. Dataloading: 0.0010 s/iter. Inference: 0.0872 s/iter. Eval: 0.0001 s/iter. Total: 0.0884 s/iter. ETA=0:02:19
[04/05 14:00:40 d2.evaluation.evaluator]: Inference done 354/1878. Dataloading: 0.0010 s/iter. Inference: 0.0869 s/iter. Eval: 0.0001 s/iter. Total: 0.0881 s/iter. ETA=0:02:14
[04/05 14:00:46 d2.evaluation.evaluator]: Inference done 411/1878. Dataloading: 0.0010 s/iter. Inference: 0.0871 s/iter. Eval: 0.0001 s/iter. Total: 0.0883 s/iter. ETA=0:02:09
[04/05 14:00:51 d2.evaluation.evaluator]: Inference done 470/1878. Dataloading: 0.0010 s/iter. Inference: 0.0867 s/iter.

In [14]:
# empty the used GPU memory
#torch.cuda.empty_cache()